# Notebook to store customers records (JSON) into bronze.customers table

Define constant variables (location, target entity)

In [0]:
%run ../../config/variables

In [0]:
TARGET_TABLE="brz_customers"

### Getting the customers (parquet)

In [0]:
df_customers=spark.read.parquet(customers_volume_path)
df_customers.createOrReplaceTempView("customers")

### Write into bronze.customers

In [0]:
spark.sql(f"""
  MERGE INTO {catalog_name}.{bronze_schema_name}.{TARGET_TABLE} AS target
  USING customers AS source
  ON target.customer_id = source.customer_id
  WHEN MATCHED AND (
      target.name    <> source.name OR
      target.phone   <> source.phone OR
      target.email   <> source.email OR
      target.address <> source.address
  ) THEN
    UPDATE SET
      target.name    = source.name,
      target.phone   = source.phone,
      target.email   = source.email,
      target.address = source.address
  WHEN NOT MATCHED THEN
    INSERT (
      customer_id,
      name,
      phone,
      email,
      address
    )
    VALUES (
      source.customer_id,
      source.name,
      source.phone,
      source.email,
      source.address
    )
""")